In [1]:
import sagemaker
import boto3
from sagemaker import get_execution_role

sagemaker_session = sagemaker.LocalSession()
iam = boto3.client('iam')

# Get a SageMaker-compatible role used by this Notebook Instance.
role = get_execution_role()

# S3 prefix
#bucket = sagemaker_session.default_bucket()
dependancy_path = "dependencies.py"
train_data = "s3://empid-bucketname/housing_train_data.csv"

In [2]:
from sagemaker.sklearn.estimator import SKLearn

FRAMEWORK_VERSION = "0.23-1"
script_path = 'sklearn_entry_point_Latest.py'

sklearn_preprocessor = SKLearn(
    entry_point=script_path,
    dependencies=['dependencies.py'],
    role=role,
    framework_version=FRAMEWORK_VERSION,
    instance_type="ml.m4.xlarge",
    sagemaker_session=sagemaker_session)

In [3]:
sklearn_preprocessor.fit({'train': train_data})

Creating cj1jqba4u0-algo-1-y02q3 ... 
Creating cj1jqba4u0-algo-1-y02q3 ... done
Attaching to cj1jqba4u0-algo-1-y02q3
cj1jqba4u0-algo-1-y02q3 | 2021-05-20 08:36:33,876 sagemaker-containers INFO     Imported framework sagemaker_sklearn_container.training
cj1jqba4u0-algo-1-y02q3 | 2021-05-20 08:36:33,880 sagemaker-training-toolkit INFO     No GPUs detected (normal if no gpus installed)
cj1jqba4u0-algo-1-y02q3 | 2021-05-20 08:36:33,892 sagemaker_sklearn_container.training INFO     Invoking user training script.
cj1jqba4u0-algo-1-y02q3 | 2021-05-20 08:36:34,046 sagemaker-training-toolkit INFO     No GPUs detected (normal if no gpus installed)
cj1jqba4u0-algo-1-y02q3 | 2021-05-20 08:36:34,061 sagemaker-training-toolkit INFO     No GPUs detected (normal if no gpus installed)
cj1jqba4u0-algo-1-y02q3 | 2021-05-20 08:36:34,077 sagemaker-training-toolkit INFO     No GPUs detected (normal if no gpus installed)
cj1jqba4u0-algo-1-y02q3 | 2021-05-20 08:36:34,090 sagemaker-training-toolkit INFO     In

In [4]:
# Define a SKLearn Transformer from the trained SKLearn Estimator
transformer = sklearn_preprocessor.transformer(
    instance_count=1, 
    instance_type='ml.m4.xlarge',
    assemble_with = 'Line',
    accept = 'text/csv')

In [5]:
# Preprocess training input
transformer.transform(train_data, content_type="text/csv")
print("Waiting for transform job: " + transformer.latest_transform_job.job_name)
transformer.wait()
preprocessed_train = transformer.output_path

Attaching to 9d5mld2tw5-algo-1-jjrez
9d5mld2tw5-algo-1-jjrez | 2021-05-20 08:37:32,392 INFO - sagemaker-containers - No GPUs detected (normal if no gpus installed)
9d5mld2tw5-algo-1-jjrez | 2021-05-20 08:37:32,395 INFO - sagemaker-containers - No GPUs detected (normal if no gpus installed)
9d5mld2tw5-algo-1-jjrez | 2021-05-20 08:37:32,396 INFO - sagemaker-containers - nginx config: 
9d5mld2tw5-algo-1-jjrez | worker_processes auto;
9d5mld2tw5-algo-1-jjrez | daemon off;
9d5mld2tw5-algo-1-jjrez | pid /tmp/nginx.pid;
9d5mld2tw5-algo-1-jjrez | error_log  /dev/stderr;
9d5mld2tw5-algo-1-jjrez | 
9d5mld2tw5-algo-1-jjrez | worker_rlimit_nofile 4096;
9d5mld2tw5-algo-1-jjrez | 
9d5mld2tw5-algo-1-jjrez | events {
9d5mld2tw5-algo-1-jjrez |   worker_connections 2048;
9d5mld2tw5-algo-1-jjrez | }
9d5mld2tw5-algo-1-jjrez | 
9d5mld2tw5-algo-1-jjrez | http {
9d5mld2tw5-algo-1-jjrez |   include /etc/nginx/mime.types;
9d5mld2tw5-algo-1-jjrez |   default_type application/octet-stream;
9d5mld2tw5-algo-1-jjre

In [6]:
from sagemaker.model import Model
from sagemaker.pipeline import PipelineModel
import boto3
from time import gmtime, strftime

timestamp_prefix = strftime("%Y-%m-%d-%H-%M-%S", gmtime())

scikit_learn_inferencee_model = sklearn_preprocessor.create_model()


model_name = "inference-" + timestamp_prefix
endpoint_name = "inference-ep-" + timestamp_prefix
sm_model = PipelineModel(
    name=model_name, role=role, models=[scikit_learn_inferencee_model]
)

sm_model.deploy(initial_instance_count=1, instance_type="ml.m4.xlarge", endpoint_name=endpoint_name)

-------------!

In [7]:
from sagemaker.predictor import json_serializer, csv_serializer, json_deserializer, Predictor
#from sagemaker.content_types import CONTENT_TYPE_CSV, CONTENT_TYPE_JSON

csv_serializer = sagemaker.serializers.CSVSerializer(content_type="text/csv")
data = """-122.26,37.46,26,5067,750,1996,728,7.0001,NEAR OCEAN"""

sagemaker_session = sagemaker.Session()

predictor = Predictor(
    endpoint_name=endpoint_name,
    sagemaker_session=sagemaker_session,
    serializer=csv_serializer)


print(predictor.predict(data))

b'{"instances": [{"features": [-1.3408764930120565, 0.8514634482951091, -0.21099315585118414, 1.1430410231864847, 0.5257730939190086, 0.5164773116405509, 0.6146760713962672, 1.6402561279658028, 0.5819441198401629, -0.030616763948385764, -1.0052566158906469, 0.0, 0.0, 0.0, 0.0, 1.0]}]}'
